In [99]:
import tweepy
import pandas as pd 
import numpy as np

## Autenticação 

In [5]:
consumer_key= 'drhztkGNhICDt3cwBGXi7zzQk'
consumer_secret= 'FDeCBlSi4qIbR9oPsf3tphQjCO6HqZfYANWceYIdJDZHsOaabi'
access_token= '207778119-E9W1kqRXBbNeU2n8vcA0OoMn9XZpE9yvmZrD8gUx'
access_token_secret= 'dncbYQQrh5jlJ97OmgxeaGgqOFiC7jnNpXdPngWMCRtDE'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


## Query API V1

In [6]:
# Define the search term and the date_since date as variables
search_words = "DisneyPlus"
date_since = "2020-11-03"

tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang="pt",
              since=date_since).items(100)

## Resultados: Cursor to List

In [7]:
todos = list(tweets)

## Extract DataFrame original

In [2]:
def extract_timeline_as_df(timeline_list):
    columns = set()
    allowed_types = [str, int]
    tweets_data = []
    for status in timeline_list:
        status_dict = dict(vars(status))
        keys = status_dict.keys()
        single_tweet_data = {"user": status.user.screen_name, "author": status.author.screen_name}
        for k in keys:
            try:
                v_type = type(status_dict[k])
            except:
                v_type = None
            if v_type != None:
                if v_type in allowed_types:
                    single_tweet_data[k] = status_dict[k]
                    columns.add(k)
        tweets_data.append(single_tweet_data)


    header_cols = list(columns)
    header_cols.append("user")
    header_cols.append('author')
    df = pd.DataFrame(tweets_data, columns=header_cols)
    return df

## Extract DataFrame modificado

In [85]:
def extract_tweets(tweets_originais):

    # Filtra os dados que eu vou tirar
    columns = set()
    allowed_types = [str, int]

    #vai virar uma lista de dicionários que vai ser transformada em df
    tweets_data = []


    for cada_tweet in tweets_originais:
        #transforma cada_tweet em um dicionario
        cada_tweet_dict = dict(vars(cada_tweet))
        #Acha a lista de keys que tem em cada_tweet
        keys = cada_tweet_dict.keys()
        #Cria um dicionario com duas chaves que não tem na lista acima, e ja coloca o conteúdo nelas
        single_tweet_data = {"screen_name": cada_tweet.user.screen_name,  
        "user_id": cada_tweet.user.id, 
        "verified":cada_tweet.user.verified, 
        "followers": cada_tweet.user.followers_count, 
        "following": cada_tweet.user.friends_count, 
        'created_at': cada_tweet.created_at, 
        'mentions':cada_tweet.entities['user_mentions'],
        'retweets': cada_tweet.retweeted_status.user}

        for k in keys:

            try:
                #Define o tipo de conteúdo dentro da chave
                v_type = type(cada_tweet_dict[k])

            except:
                # Se o tipo de conteúdo for nulo, ou seja, o trem de cima não funcionar, coloca ele como None
                v_type = None

            #Depois de definir o tipo de conteúdo para uma chave k, vamos fazer alguma coisa com ele
            if v_type != None:
                #Então, se o conteúdo da chave tiver dos tipos permitidos

                if v_type in allowed_types:
                    #Vamos adicionar este conteúdo no dicionário criado lá em cima
                    single_tweet_data[k] = cada_tweet_dict[k]

                    columns.add(k)

        #Adiciona o dicionário final criado a uma lista            
        tweets_data.append(single_tweet_data)

    #Transforma as colunas que estavam em forma de set em lista
    header_cols = list(columns)
    #Adiciona o chave user, já que ele tá fora das chaves principais
    header_cols.append("screen_name")
    header_cols.append("user_id")
    header_cols.append("verified")
    header_cols.append("followers")
    header_cols.append("following")
    header_cols.append('created_at')
    header_cols.append('mentions')
    header_cols.append('retweets')
    
    #Cria o dataframe
    df = pd.DataFrame(tweets_data, columns=header_cols)
    return df

In [111]:
def extract_tweets2(tweets_originais):

    # Filtra os dados que eu vou tirar
    columns = set()

    #vai virar uma lista de dicionários que vai ser transformada em df
    tweets_data = []


    for cada_tweet in tweets_originais:
        #Acha a lista de keys que tem em cada_tweet
        keys = cada_tweet._json.keys()
        single_data = {}

        for k in keys:
            try:
                cada_tweet._json.get(k)
                single_data[k] = cada_tweet._json.get(k)

            except:
                single_data[k] = np.nan

            columns.add(k)
        

        #Adiciona o dicionário final criado a uma lista            
        tweets_data.append(single_data)


    #Transforma as colunas que estavam em forma de set em lista
    header_cols = list(columns)
    
    #Cria o dataframe
    df = pd.DataFrame(tweets_data, columns=header_cols)
    return df

In [156]:
## Funcao final

columns = set()
  
tweets_data = []
  
for tweet in todos:
    keys = tweet._json.keys()
    single_data = {}
      
      
    for k in keys:
        try:
            single_data[k] = tweet._json.get(k)
                
        except Exception as e:
            print(e)
            single_data[k] = np.nan
            
        columns.add(k)
    tweets_data.append(single_data)    
  

header_cols = list(columns)
df = pd.DataFrame(tweets_data, columns=header_cols)
df.head()
 
df.to_csv(r'C:\Users\amand\Documents\Jupyter Notebooks\DisneyPlus\Data\Teste1000_2.csv')



TypeError: 'Cursor' object is not iterable